In [2]:
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Nomenclature

### LCOE

$l_1$ : lcoe for once through cycle

### Front End

$f_1$ : lcoe for front end with

    - $u_1$ : Raw Ore
    - $b_1$ : Fuel treatment as
        - Conversion
        - Enrichment
        - Fabrication

$f_1 = u_1 + b_1$

### Reactor

$k_1$ : Capital cost 
$m_1$ : Operation and Maintenance

### Back End

$d_1$ : Back end cost with

    - Above ground storage
    - Final geological sequestration


# Input Eco Data

In [3]:
mit_data = False
BU = 50

In [4]:
# discount rate
dr_disc = 0.07
dr_cont = np.log(1+dr_disc)

# Number taken from Du Parson Update Costs of Nuclear - Excel Table For Investment
tax_rate = 0.37

### Front End

In [5]:
# Uranium Ore [$/kgHM]
cost_UNat = 80
# Depleted Uranium [$/kgHM]
cost_UDep = 10
# Conversion of Nat U [$/kgHM]
cost_UConv = 10
# Enrichment of Nat U [$/SWU]
cost_UEnr = 160
# Fabrication of UOX [$/kgHM] => "From Natural Uranium ??"
cost_UFab = 250

### Reactor

In [6]:
# LWR Capital overnigh [$ / kWe]
cost_occ_lwr = 4000
# LWR annual incremental capital  [$ / kWe] ??
cost_ann_incremental_capital = 40
# dismantling [$]
cost_dismantling = 0.175 * cost_occ_lwr
# om fix [$ / kWe]
cost_om_fix = 56.44
# om fix [$ / kWh]
cost_om_var = 0.42e-3

### Waste Costs

In [7]:
# Interim storage of UOx [$/kgiHM]
cost_storage_uox = 200
# Var Cost of disposal [$/kg-iHM]
# cost_disposal_uox = 463
# Var Cost of disposal equivalent fee [$/kWh]
cost_disposal_uox_eq = 1e-3

### Reprocessing

In [8]:
# UOX Purex [$/kg] => Why 4000 in the paper ??
cost_purex = 1600
# Disposal of HLW from UOX (PUREX) [$/kg iHN]
cost_disposal_hlw_from_uox = 185

### Reactor Technic data

In [9]:
# Reactor data
pwr_lifetime = 60
pwr_thermal_efficiency = 0.33
pwr_capacityfactor = 0.85
pwr_sf_cooling = 5

# De Roo Data
if mit_data:
    pwr_burnup = 50
    pwr_fuel_campaign_length = 1.5
    pwr_batch = 3
    pwr_fuel_cycle_length = pwr_fuel_campaign_length * pwr_batch
    # Specific Power [W/g]
    pwr_specificpower = pwr_burnup*1e9 / (pwr_fuel_cycle_length*365.25*1e6)
# User Data
else:
    pwr_burnup = BU
    # Specific Power [W/g] - Here MIT SP
    pwr_specificpower = 30.420564301467792
    pwr_fuel_cycle_length = pwr_burnup*1e9 / (pwr_specificpower*365.25*1e6)

# Mass of HN to have a nominal electric power of 1GWe [kg]
pwr_mass_hn_for_1GWe = pwr_capacityfactor / (pwr_thermal_efficiency * pwr_specificpower) *1e6
# Effective electric power produced by 1 kg of Uenr [kWe]
pwr_effective_elec_1kguox = 1e6 * pwr_capacityfactor / (pwr_mass_hn_for_1GWe)
# Effective energy generated by 1kg of UOX - Levelized ! - [kWh]
pwr_effective_elec_1kguox_L = (pwr_effective_elec_1kguox * 8766 * (1 - np.exp(-dr_cont*pwr_fuel_cycle_length))/dr_cont)
# Effective energy generated by a mass of UOX providing 1GWe [kWh]
pwr_effective_elec_core_L = (pwr_effective_elec_1kguox * pwr_mass_hn_for_1GWe * 8766 * (1 - np.exp(-dr_cont*pwr_lifetime))/dr_cont)

psy_L = pwr_effective_elec_1kguox_L
phi_L = pwr_effective_elec_core_L

# PWR - UOX Burn-Up versus Enrichment relation = From Scenario.root file
def Enr(bu):
    p0 = 0.0134444
    p1 = 0.000562051
    p2 = 1.35931e-06
    return p0 + p1*bu + p2*bu**2

# PWR - UOX UTS for 1 kg of Uenr
def swu(uapp,uenr):
    p0 = -0.4975
    p1 =  -13.08
    p2 =      28
    p3 =   2.705
    p4 =    0.03
    p5 =   -3.44
    return p0 + p1*uapp + p2*uapp**2 + p3*uenr + p4*uenr**2 + p5*uapp*uenr

# Once-Through Cycle

### Up-front technical data

In [10]:
# mass loss during conversion, enrichment and fabrication
loss = 0.002

# enrichissement massique !!!
enr_unat = 0.007112
enr_uapp = 0.0029
# Enr calculation
if mit_data:
    enr_uenr = 0.045
else:
    enr_uenr = Enr(pwr_burnup)

# Ratio Product / Feed
ratio_mp_mf = (enr_unat - enr_uapp)/(enr_uenr - enr_uapp) * (1 - loss)
# 3 stages for fuel fabrication 
mass_unat_per_kg_uf6 = 1/(1-loss)
mass_uf6_per_kg_uenr = 1 / ratio_mp_mf
mass_uenr_per_kg_uox = 1/(1-loss)
# Mass of unat for 1 kg of UOx
mass_unat_per_kg_uox = mass_unat_per_kg_uf6 * mass_uf6_per_kg_uenr * mass_uenr_per_kg_uox

# SWU calculation
if mit_data:
    swu_enrichment = 6.37
else:
    swu_enrichment = swu(100*enr_uapp,100*enr_uenr) * 1/(1-loss)


### Up-front eco calculation

In [11]:
u1 = (cost_UNat * mass_unat_per_kg_uox * 1/(1+dr_disc)**(-2)) / psy_L

conv = (cost_UConv * mass_uf6_per_kg_uenr*mass_uenr_per_kg_uox) * 1/(1+dr_disc)**(-1.5)
enr =  (cost_UEnr * swu_enrichment) * 1/(1+dr_disc)**(-1)
fab = (cost_UFab * 1) * 1/(1+dr_disc)**(-0.5)

b1 = (conv + enr + fab)/psy_L

f1 = u1 + b1


### Capital MIT method

In [16]:
#construction_year    = [-10 ,-9  ,-8  ,-7  ,-6  ,-5  ,-4  ,-3  ,-2  ,-1  ]
#construction_profile = [0.02,0.05,0.08,0.14,0.19,0.18,0.15,0.10,0.07,0.02]

construction_year    = [-5   ,-4   ,-3   ,-2   ,-1  ]
construction_profile = [0.095,0.250,0.310,0.250,0.095]

if sum(construction_profile) != 1:
    print('ERROR ! : ',sum(construction_profile),' != 0')

# Cash Flow Profile
fig = go.Figure()
fig.add_trace(go.Bar(name="Construction",x=construction_year,y=construction_profile))
fig.update_layout(barmode='stack',
                  title="Profile de coûts",
                  xaxis_title="Temps (année)",
                  yaxis_title="Dépense annuelle (€)")
#fig.show()

# TERM 1 - Sum for construction schedule
ct = len(construction_year)
construction_occ_factor = 0
for i in range(ct):
    construction_occ_factor += construction_profile[i] * 1/(1+dr_disc)**(i+1-ct)
construction_tic = cost_occ_lwr * construction_occ_factor

# TERM 2 - Depreciation => Correct BUT to understand
dt = 16
construction_depreciation_factor = 0
# Wikipedia numbers for 15 years MACRS depreciation schedule
Dt = [5,9.5,8.55,7.7,6.93,6.23,5.90,5.9,5.91,5.9,5.91,5.9,5.91,5.9,5.91,2.95]
for i in range(dt):
    #Dt = 1 - 1/16*(i+1)
    construction_depreciation_factor += Dt[i]/100 * tax_rate * 1/(1+dr_disc)**(i+1)
    #print("%.2f" % Dt[i],tax_rate,i+1,"%.2f" % (1/(1+dr)**(i+1)),"%.2f" % construction_depreciation_factor,sep='\t')
construction_depreciation = cost_occ_lwr * construction_depreciation_factor
    
# TERM 3 - Annual incremental capital cost -> What is this ?
construction_aic_factor = 0
for t in range(pwr_lifetime):
    construction_aic_factor += 1/(1+dr_disc)**(t+1)
construction_aic = cost_ann_incremental_capital * (1 - tax_rate) * construction_aic_factor

# TERM 4 - Decommissioning
dismantling = cost_dismantling * (1 - tax_rate) * 1/(1+dr_disc)**pwr_lifetime

# Sum of terms
k1 = (construction_tic -construction_depreciation + dismantling + construction_aic) * 1e6 / (phi_L*(1-tax_rate))


### Capital PMT method : PMT / Annual Energie

### Operation § Maintenance (including pool)

In [29]:
om_factor = 0
for t in range(pwr_lifetime):
    om_factor += 1/(1+dr_disc)**(t+1)

m1 = cost_om_fix * 1e6 * om_factor / phi_L + cost_om_var

### Waste Managment

In [30]:
d1 = cost_storage_uox * 1/(1+dr_disc)**(pwr_fuel_cycle_length+pwr_sf_cooling) / psy_L + cost_disposal_uox_eq
d1
pwr_fuel_cycle_length

4.5

### LCOE Global

In [31]:
#A Décomposer l'écriture :

l1 = f1 + k1 + m1 + d1

print('######################################')
print('MIT : ',mit_data)
print('\tBU : ',pwr_burnup,' GWd/t')
print('######################################')
print('f1 = ',"%.2f" % (f1*1000),' mill$/kWh')
print('\tu1 = ',"%.2f" % (u1*1000),' mill$/kWh')
print('\tb1 = ',"%.2f" % (b1*1000),' mill$/kWh')
print('######################################')
print('k1 = ',"%.2f" % (k1*1000),' mill$/kWh')
print('######################################')
print('m1 = ',"%.2f" % (m1*1000),' mill$/kWh')
print('######################################')
print('d1 = ',"%.2f" % (d1*1000),' mill$/kWh')
print('######################################')
print('l1 = ',"%.2f" % (l1*1000),' mill$/kWh')


######################################
MIT :  False
	BU :  50  GWd/t
######################################
f1 =  6.96  mill$/kWh
	u1 =  2.69  mill$/kWh
	b1 =  4.27  mill$/kWh
######################################
k1 =  59.30  mill$/kWh
######################################
m1 =  7.74  mill$/kWh
######################################
d1 =  1.31  mill$/kWh
######################################
l1 =  75.31  mill$/kWh


# Twice through cycle

### PWR UOX - Front End

In [15]:
f21 = f1

### PWR - UOX - Reactor

In [16]:
k21 = k1
m21 = m1

### PWR - UOX - Back End

In [17]:
# Reprocessing from Purex
s21 = cost_purex *1/(1+dr_disc)**(pwr_fuel_cycle_length + pwr_sf_cooling) / psy_L
# Waste for PF and MA
w21 = cost_disposal_hlw_from_uox * (1+dr_disc)**(pwr_sf_cooling)/(1+dr_disc)**(pwr_fuel_cycle_length+pwr_sf_cooling) / psy_L
# Credit for reprocessed uranium
def mass_u_reprocessed_per_ikg_u

u21b




###########################################################
print('s21 = ',"%.2f" % (s21*1000),' mill$/kWh')
print('w21 = ',"%.2f" % (w21*1000),' mill$/kWh')

SyntaxError: invalid syntax (<ipython-input-17-c8f867f0089e>, line 6)

In [ ]:
step = 0.0001
ycsum = 0
tc = np.arange(0,10,step)
yc = np.exp(-dr*tc)
ycsum = yc.sum() * step

ydsum = 0
td = np.linspace(0.5,9.5,10)
yd = 1/(1+dr)**td
ydsum = yd.sum()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=tc, y=yc,
    name='continuous',
    mode='lines'))
fig.add_trace(go.Scatter(
    x=td, y=yd,
    name='discrete',
    mode='markers'))
fig.update_layout(title="actualisation",
                  xaxis_title="Temps (année)",
                  yaxis_title="Valeur F.A.")
#fig.show()

dcc = (1 - np.exp(-dr*10))/dr

#print(ydsum)
#print(ycsum)
#print(dcc)